In [1]:
import pandas as pd
import numpy as np
import math 
import matplotlib 
from sklearn.metrics.pairwise import cosine_similarity

## Loading Dataset

In [2]:
ratings = pd.read_csv(r"D:\FastSemesters\semester6\Recommender_System(E)\k201716\ml-latest-small\ratings.csv")
movies = pd.read_csv(r"D:\FastSemesters\semester6\Recommender_System(E)\k201716\ml-latest-small\movies.csv")

## Preprocessing

In [3]:
#after looking at all datasets in the ml_latest files, for our tasks now, we will use ratings and movies dataset.
#merging the dataset using join concept from db on movieid as pk.
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [4]:
mergedData = pd.merge(ratings, movies, on='movieId', how='inner')  
finalData = mergedData.pivot_table(index='userId', columns='movieId', values='rating') #creating matrix
finalData.tail(5)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
609,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
610,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
user_mean = finalData.mean(axis=1)
finalData= finalData.sub(user_mean, axis=0)
finalData.tail(2)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
609,-0.270270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.72973,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
610,1.311444,NaN,NaN,NaN,NaN,1.311444,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Similarity

In [6]:
similarity = pd.DataFrame(cosine_similarity(finalData.fillna(0)), 
                          index = finalData.index,
                          columns = finalData.index )
similarity.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.001265,0.000553,0.048419,0.021847,-0.045497,-0.006200,0.047013,0.019510,-0.008754,...,0.018127,-0.017172,-0.015221,-0.037059,-0.029121,0.012016,0.055261,0.075224,-0.025713,0.010932
2,0.001265,1.000000,0.000000,-0.017164,0.021796,-0.021051,-0.011114,-0.048085,0.000000,0.003012,...,-0.050551,-0.031581,-0.001688,0.000000,0.000000,0.006226,-0.020504,-0.006001,-0.060091,0.024999
3,0.000553,0.000000,1.000000,-0.011260,-0.031539,0.004800,0.000000,-0.032471,0.000000,0.000000,...,-0.004904,-0.016117,0.017749,0.000000,-0.001431,-0.037289,-0.007789,-0.013001,0.000000,0.019550
4,0.048419,-0.017164,-0.011260,1.000000,-0.029620,0.013956,0.058091,0.002065,-0.005874,0.051590,...,-0.037687,0.063122,0.027640,-0.013782,0.040037,0.020590,0.014628,-0.037569,-0.017884,-0.000995
5,0.021847,0.021796,-0.031539,-0.029620,1.000000,0.009111,0.010117,-0.012284,0.000000,-0.033165,...,0.015964,0.012427,0.027076,0.012461,-0.036272,0.026319,0.031896,-0.001751,0.093829,-0.000278


In [7]:
active_user = int(input())
print("active user is user:",active_user)
k = int(input())
print("K neighbors:",k)
movietorate = str(input())

1
active user is user: 1
5
K neighbors: 5
jumanji


In [8]:
similar_users = similarity[active_user].nlargest(k).index
print(f'similar users are:',similar_users)

similar users are: Int64Index([1, 301, 597, 414, 477], dtype='int64', name='userId')


In [9]:
def predict_rating(user_id, movie_id,similar_users):
    ratings = finalData.loc[similar_users, movie_id].dropna()
    
    if ratings.empty:
        return user_means[user_id]
    else:
        similarities = similarity.loc[user_id, ratings.index]
        weighted_ratings = similarities * ratings
        return user_means[user_id] + weighted_ratings.sum() / similarities.sum()

In [19]:
# user_rated_movies = ratings.loc[ratings['userId'] == 1, 'movieId']
# unrated_movies = set(ratings['movieId']) - set(user_rated_movies)
# # unrated_movies

In [18]:
user_watched = finalData[finalData.index == active_user].dropna(axis=1, how='all')

In [ ]:
activeuser_mean= finalData[finalData!=0].loc[active_user].mean()
activeuser_mean